In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

class Task:
    def __init__(self, name, category, estimated_cost):
        self.name = name
        self.category = category
        self.estimated_cost = estimated_cost
        self.actual_cost = 0.0

    def update_actual_cost(self, cost):
        self.actual_cost = cost

    def variance(self):
        return self.actual_cost - self.estimated_cost

    def status(self):
        if self.actual_cost > self.estimated_cost * 1.2:
            return "⚠️ Over Budget"
        elif self.actual_cost < self.estimated_cost * 0.8:
            return "⚠️ Under Budget"
        else:
            return "✓ On Budget"

    def suggestion(self):
        if self.status() == "⚠️ Over Budget":
            return "Consider reviewing materials, labor, or outsourcing options."
        elif self.status() == "⚠️ Under Budget":
            return "Check if quality is being compromised."
        else:
            return "Costs are on track."

class AIBudgetAnalyzer:
    def __init__(self):
        self.tasks = []
        # ✅ Fix: Define data types to avoid future warning
        self.past_data = pd.DataFrame({
            "Estimated": pd.Series(dtype='float'),
            "Actual": pd.Series(dtype='float')
        })

    def add_task(self, name, category, estimated_cost):
        task = Task(name, category, estimated_cost)
        self.tasks.append(task)

    def enter_actual_costs(self):
        for task in self.tasks:
            cost = get_valid_float(f"Enter actual cost for '{task.name}' (₹): ")
            task.update_actual_cost(cost)

            # No warning with explicitly typed DataFrame
            new_row = pd.DataFrame([{
                "Estimated": task.estimated_cost,
                "Actual": task.actual_cost
            }])
            self.past_data = pd.concat([self.past_data, new_row], ignore_index=True)

    def predict_cost(self, new_estimated):
        if len(self.past_data) < 3:
            return new_estimated  # not enough data to predict
        X = self.past_data["Estimated"].values.reshape(-1, 1)
        y = self.past_data["Actual"].values
        model = LinearRegression()
        model.fit(X, y)
        predicted = model.predict(np.array([[new_estimated]]))[0]
        return round(predicted, 2)

    def generate_report(self):
        print("\n📊 === AI Budget Analysis Report ===")
        total_est = total_act = 0

        for task in self.tasks:
            print(f"\n🛠️ Task: {task.name} ({task.category})")
            print(f"   Estimated Cost: ₹{task.estimated_cost}")
            print(f"   Actual Cost: ₹{task.actual_cost}")
            print(f"   Variance: ₹{task.variance()}")
            print(f"   Status: {task.status()}")
            print(f"   Suggestion: {task.suggestion()}")

            ai_prediction = self.predict_cost(task.estimated_cost)
            print(f"   🔮 AI Predicted Cost: ₹{ai_prediction}")

            total_est += task.estimated_cost
            total_act += task.actual_cost

        print("\n📌 === Summary ===")
        print(f"Total Estimated: ₹{total_est}")
        print(f"Total Actual: ₹{total_act}")
        print(f"Total Variance: ₹{total_act - total_est}")
        print("Project Status:", "Over Budget" if total_act > total_est else "Under Budget" if total_act < total_est else "On Budget")

# -----------------------
# Input Validation Utils
# -----------------------

def get_valid_int(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("❌ Please enter a valid number.")

def get_valid_float(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("❌ Please enter a valid amount (e.g., 5000.75).")

# -----------------------
# Main Execution
# -----------------------

if __name__ == "__main__":
    analyzer = AIBudgetAnalyzer()
    num = get_valid_int("Enter number of tasks: ")

    for _ in range(num):
        name = input("\nEnter task name: ")
        category = input("Enter task category (e.g., Labor, Materials, Tech): ")
        est_cost = get_valid_float("Enter estimated cost (₹): ")
        analyzer.add_task(name, category, est_cost)

    analyzer.enter_actual_costs()
    analyzer.generate_report()
    

Enter number of tasks:  3

Enter task name:  app development
Enter task category (e.g., Labor, Materials, Tech):  tech
Enter estimated cost (₹):  70000

Enter task name:  graphic design
Enter task category (e.g., Labor, Materials, Tech):  creation
Enter estimated cost (₹):  20000

Enter task name:  marketing
Enter task category (e.g., Labor, Materials, Tech):  promotion
Enter estimated cost (₹):  30000
Enter actual cost for 'app development' (₹):  90000
Enter actual cost for 'graphic design' (₹):  19000
Enter actual cost for 'marketing' (₹):  20000



📊 === AI Budget Analysis Report ===

🛠️ Task: app development (tech)
   Estimated Cost: ₹70000.0
   Actual Cost: ₹90000.0
   Variance: ₹20000.0
   Status: ⚠️ Over Budget
   Suggestion: Consider reviewing materials, labor, or outsourcing options.
   🔮 AI Predicted Cost: ₹88428.57

🛠️ Task: graphic design (creation)
   Estimated Cost: ₹20000.0
   Actual Cost: ₹19000.0
   Variance: ₹-1000.0
   Status: ✓ On Budget
   Suggestion: Costs are on track.
   🔮 AI Predicted Cost: ₹12714.29

🛠️ Task: marketing (promotion)
   Estimated Cost: ₹30000.0
   Actual Cost: ₹20000.0
   Variance: ₹-10000.0
   Status: ⚠️ Under Budget
   Suggestion: Check if quality is being compromised.
   🔮 AI Predicted Cost: ₹27857.14

📌 === Summary ===
Total Estimated: ₹120000.0
Total Actual: ₹129000.0
Total Variance: ₹9000.0
Project Status: Over Budget
